<a href="https://colab.research.google.com/github/vikpy/AISem3/blob/master/HW/HomeWork_1_Classification_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Difference between one hot encoding and label encoding for logistic regression

In [ ]:
!ls -al

total 3326004
drwxr-xr-x 1 root root      4096 Jul  5 13:39 .
drwxr-xr-x 1 root root      4096 Jul  5 12:50 ..
-rw-r--r-- 1 root root  26567651 Dec 11  2019 application_test.csv
-rw-r--r-- 1 root root   6091687 Jul  5 13:22 application_test.csv.zip
-rw-r--r-- 1 root root 166133370 Dec 11  2019 application_train.csv
-rw-r--r-- 1 root root  37847529 Jul  5 13:22 application_train.csv.zip
-rw-r--r-- 1 root root 375592889 Dec 11  2019 bureau_balance.csv
-rw-r--r-- 1 root root  59574401 Jul  5 13:22 bureau_balance.csv.zip
-rw-r--r-- 1 root root 170016717 Dec 11  2019 bureau.csv
-rw-r--r-- 1 root root  38550359 Jul  5 13:22 bureau.csv.zip
drwxr-xr-x 1 root root      4096 Jun 26 16:26 .config
-rw-r--r-- 1 root root 424582605 Dec 11  2019 credit_card_balance.csv
-rw-r--r-- 1 root root 101435187 Jul  5 13:22 credit_card_balance.csv.zip
-rw-r--r-- 1 root root     37383 Jul  5 13:22 HomeCredit_columns_description.csv
-rw-r--r-- 1 root root 723118349 Dec 11  2019 installments_payments.csv
-rw-r--r

In [ ]:
import json
token = {"username":"vikraantpy","key":"8eb2028c6e7713733718b019327a5204"}   # Key removed for confidentiality 
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c home-credit-default-risk -p /content/

installments_payments.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
HomeCredit_columns_description.csv: Skipping, found more recently modified local copy (use --force to force download)
previous_application.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
bureau_balance.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
bureau.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
application_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
application_test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
credit_card_balance.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
POS_CASH_balance.csv

In [50]:
!pip install pyforest
from pyforest import * 

In [ ]:
!unzip \*.zip

In [51]:
# Memory compression https://www.kaggle.com/gemartin/load-data-reduce-memory-usage 

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [52]:
def find_nulls(df):
  return df.isnull().sum()[df.isnull().sum()>0].sort_values(ascending=False) 

In [53]:
path_to_file = "/content/"
print('-' * 80)
print('train')
train = import_data(path_to_file + 'application_train.csv')
print("TRAIN")
print(find_nulls(train))

print('-' * 80)
print('test')
test = import_data(path_to_file + 'application_test.csv')
print("TEST")
print(find_nulls(test))

print('-' * 80)
print('bureau_balance')
bureau_balance = import_data(path_to_file + 'bureau_balance.csv')
print("BUREAU BALANCE")
print(find_nulls(bureau_balance))

print('-' * 80)
print('bureau')
bureau = import_data(path_to_file + 'bureau.csv')
print("BUREAU")
print(find_nulls(bureau))

print('-' * 80)
print('credit_card_balance')
credit_card = import_data(path_to_file + 'credit_card_balance.csv')
print("Credit Card Balance")
print(find_nulls(credit_card))

print('-' * 80)
print('installments_payments')
installments = import_data(path_to_file + 'installments_payments.csv')
print("Installments work")
print(find_nulls(installments) )

print('-' * 80)
print('pos_cash_balance')
pos_cash = import_data(path_to_file + 'POS_CASH_balance.csv')
print("POS Cash Balance")
print(find_nulls(pos_cash) )

print('-' * 80)
print('previous_application')
previous_app = import_data(path_to_file + 'previous_application.csv')
print("Previous Application")
print(find_nulls(previous_app) )
print('-' * 80)

--------------------------------------------------------------------------------
train


<IPython.core.display.Javascript object>

Memory usage of dataframe is 286.23 MB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Memory usage after optimization is: 130.62 MB
Decreased by 72.3%
Previous Application
RATE_INTEREST_PRIVILEGED     1664263
RATE_INTEREST_PRIMARY        1664263
RATE_DOWN_PAYMENT             895844
AMT_DOWN_PAYMENT              895844
NAME_TYPE_SUITE               820405
NFLAG_INSURED_ON_APPROVAL     673065
DAYS_TERMINATION              673065
DAYS_LAST_DUE                 673065
DAYS_LAST_DUE_1ST_VERSION     673065
DAYS_FIRST_DUE                673065
DAYS_FIRST_DRAWING            673065
AMT_GOODS_PRICE               385515
AMT_ANNUITY                   372235
CNT_PAYMENT                   372230
PRODUCT_COMBINATION              346
AMT_CREDIT                         1
dtype: int64
--------------------------------------------------------------------------------


In [55]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018799,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,0.020493,0.019302,0.000000,0.000000,reg oper account,block of flats,0.014900,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003542,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,0.078674,0.055786,0.003901,0.010002,reg oper account,block of flats,0.071411,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010033,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008018,-19005,-3039,-9832.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028656,-19932,-3038,-4312.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
train["Flag"] = 1
test["Flag"] = 0
data = pd.concat([train.drop("TARGET", axis=1), test], axis=0)
category_columns = data.dtypes[data.dtypes == "category"].reset_index()["index"]
category_columns

<IPython.core.display.Javascript object>

0             NAME_CONTRACT_TYPE
1                   FLAG_OWN_CAR
2                FLAG_OWN_REALTY
3                NAME_TYPE_SUITE
4            NAME_EDUCATION_TYPE
5              NAME_HOUSING_TYPE
6                OCCUPATION_TYPE
7     WEEKDAY_APPR_PROCESS_START
8              ORGANIZATION_TYPE
9             FONDKAPREMONT_MODE
10                HOUSETYPE_MODE
11            WALLSMATERIAL_MODE
12           EMERGENCYSTATE_MODE
Name: index, dtype: object

In [99]:

from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
for column in category_columns: 
  data[column] = enc.fit_transform(data[column].astype(str))


In [ ]:
find_nulls()